# A quantum algorithm for the optimization of the incident response process
The problem to address is how to find and select the minimum set of incidents, from a larger starting one, that we must undertake in order to resolve all the existing incidents, using quantum algorithms.

In [1]:
import random
import numpy as np
from Incident import Incident
from qiskit import BasicAer
from qiskit.utils import algorithm_globals
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.utils import QuantumInstance
from qiskit.algorithms import QAOA

## Creation of a little dataset of example
In order to create a little real-life example dataset of incidents, controls, threst and estimated times have to be defined. Then, all these data are merged to make an unique set of 14 incidents.

In [2]:
#First of all, I've to instanciate dictionaries of controls and incidents
controls = {
    #control_key : control_name
    1 : "Information Backup",
    2 : "Capacity Management",
    3 : "Off-Site Equipment Security",
    4 : "Physical Entry Controls",
    5 : "Restriction Of Access To Information",
    6 : "Management Of Secret Information Authentication",
    7 : "Equipment Maintenance",
    8 : "Clock Synchronization",
    9 : "Supply Facilities",
    10 : "Secret authentication and information management",
    11 : "Network Controls"
}

In [3]:
threats = {
    #threat_key : [threat_name, control_key1, control_key2, ...]
    1 : ["Denial Of Service", 1, 2],
    2 : ["Theft", 3],
    3 : ["Social Engineering", 4, 10],
    4 : ["Monitoring Errors", 5, 6],
    5 : ["Failure Of Physical Or Logical Origin", 7],
    6 : ["System Crashes Due To Resource Exhaustion", 2],
    7 : ["Abuse Of Access Privileges", 5],
    8 : ["Configuration Errors", 8],
    9 : ["Maintenance Errors", 1],
    10 : ["Inadequate Temperature Or Humidity Conditions", 9],
    11 : ["Failure Of Communication Services", 11]
}

#the list of estimated time of resolution
estimated_times = [6,24,8,40,24,8,24,2,6,72,8]

In [4]:
#penalty = max(estimated_times)+1
penalty = 73

In [5]:
#Now I've to make the dataset (of 14 incidents)
incidents = []
for _ in range(14):
    threat_key = random.randint(1,11)
    threat_list = threats.get(threat_key)
    #if the list contains just the threat name, we have to choose another threat
    while(len(threat_list) < 2):
        threat_key = random.randint(1,11)
        threat_list = threats.get(threat_key)
    threat_name = threat_list[0]
    control_key = threat_list.pop(random.randint(1,len(threat_list)-1))
    control_name = controls[control_key]
    estimated_time = estimated_times[threat_key-1]
    incident = Incident(threat_key, threat_name, control_key,control_name, estimated_time)
    incidents.append(incident)

for i in range(len(incidents)):
    print(incidents[i])

threat_id: 1, threat_name: Denial Of Service, control_id: 1, control_name: Information Backup, estimated_time: 6
threat_id: 3, threat_name: Social Engineering, control_id: 4, control_name: Physical Entry Controls, estimated_time: 8
threat_id: 2, threat_name: Theft, control_id: 3, control_name: Off-Site Equipment Security, estimated_time: 24
threat_id: 10, threat_name: Inadequate Temperature Or Humidity Conditions, control_id: 9, control_name: Supply Facilities, estimated_time: 72
threat_id: 7, threat_name: Abuse Of Access Privileges, control_id: 5, control_name: Restriction Of Access To Information, estimated_time: 24
threat_id: 4, threat_name: Monitoring Errors, control_id: 6, control_name: Management Of Secret Information Authentication, estimated_time: 40
threat_id: 9, threat_name: Maintenance Errors, control_id: 1, control_name: Information Backup, estimated_time: 6
threat_id: 8, threat_name: Configuration Errors, control_id: 8, control_name: Clock Synchronization, estimated_time: 

In order to subtract a penalty amount from every discovered threat, for every control that they affected, a dictionary it's needed to be stored to map, for each actually affected control, the correspondent recorded threats.

In [6]:
#I need to know the ffected controls
actual_controls = {}
for incident in incidents:
    control_id = incident.get_control_id()
    if actual_controls.keys().__contains__(control_id):
        threat_list = actual_controls.get(control_id)
        threat_list.append(incident.get_threat_id())
        threat_list.sort()
    else:
        actual_controls[control_id] = [incident.get_threat_id()]

print("Actual affected controls:" + str(actual_controls))

Actual affected controls:{1: [1, 9], 4: [3], 3: [2], 9: [10], 5: [4, 7], 6: [4], 8: [8], 7: [5], 11: [11], 2: [1, 6], 10: [3]}


## QUBO problem definition
Using Qiskit, every kind of  BQM is expressed as a QuadraticProgram; this means that, in order to define the QUBO problem to represent the Hamiltonian objective function to optimize, an instance of QuadraticProgram has to be generated with its linear and quadratic terms.

In [7]:
def create_QUBO_problem(linear_terms,quadratic_terms):
    qubo = QuadraticProgram()
    for i in range(1,len(linear_terms)+1):
        qubo.binary_var('x%s' % (i))

    #apply the penalty for each linear term
    for threat_list in actual_controls.values():
        for threat_id in threat_list:
            linear_terms[threat_id-1] -= 73

    qubo.minimize(linear=linear_terms,quadratic=quadratic_terms)

    return qubo

The QUBO problem is formalized as an upper-diagonal NxN matrix, where rows and columns represent the threats. The diagonal (linear) terms match the estimated resolution time of the correspondent threat, subtracted by a penalty value for each affected control (this means that if the threat i.e. x1 affected two controls, the estimated resolution time is subtracted two times by a penalty value). The upper-triangular (quadratic) part of the matrix matches, for each couple of threats that affected the same control, the double of the penalty value.

In [8]:
#generating the linear terms (biases)
linear_terms = np.array(estimated_times)

#dclaring a dictrionary for the quadratic terms (coupling weights)
quadratic = {}

#filling the dictionary
for threat_list in actual_controls.values():
    for i in threat_list:
        for j in threat_list:
            if(j>i):
                try:
                    quadratic[i,j] += 2*penalty
                except:
                    quadratic[i,j] = 2*penalty
print("LINEAR:"+str(linear_terms))
print("QUADRATIC:"+str(quadratic))

LINEAR:[ 6 24  8 40 24  8 24  2  6 72  8]
QUADRATIC:{(1, 9): 146, (4, 7): 146, (1, 6): 146}


In [9]:
qubo = create_QUBO_problem(linear_terms,quadratic)
print(qubo)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: - 140 x1 - 49 x2 - 138 x3 - 106 x4 - 49 x5 - 65 x6 - 49 x7 - 71 x8 - 67 x9
      - x10 - 65 x11 + [ 292 x2*x7 + 292 x2*x10 + 292 x5*x8 ]/2
Subject To

Bounds
 0 <= x1 <= 1
 0 <= x2 <= 1
 0 <= x3 <= 1
 0 <= x4 <= 1
 0 <= x5 <= 1
 0 <= x6 <= 1
 0 <= x7 <= 1
 0 <= x8 <= 1
 0 <= x9 <= 1
 0 <= x10 <= 1
 0 <= x11 <= 1

Binaries
 x1 x2 x3 x4 x5 x6 x7 x8 x9 x10 x11
End



## Quantum optimization algorithm
In order to find approximate solutions to combinatorial-optimization problems, QAOA has been used. This algorithm is a minimum eigen solver, which we can wrap by a MinimumEigenOptimizer to make it resolve the already formulated QUBO problem. The QUBO problem will be converted in the correspondent Ising Hamiltonian and then resolved by QAOA that will return the optimum value.

In [10]:
#inizializing the optimizer
algorithm_globals.random_seed = 10598

quantum_instance = QuantumInstance(
    BasicAer.get_backend("qasm_simulator"),
    seed_simulator=algorithm_globals.random_seed,
    seed_transpiler=algorithm_globals.random_seed,
)

qaoa_mes = QAOA(quantum_instance=quantum_instance, initial_point=[0.0, 0.0])

In [11]:
#using the minimum eigen solver to istanciate the minimum eigen optimizer
qaoa = MinimumEigenOptimizer(qaoa_mes)

In [12]:
print("Wait.......")
qaoa_result = qaoa.solve(qubo)
print(qaoa_result)

Wait.......
optimal function value: -679.0
optimal value: [1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1.]
status: SUCCESS


In [13]:
print("QUBO variable order:", [var.name for var in qaoa_result.variables])
for s in qaoa_result.samples:
    print(s)

QUBO variable order: ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11']
SolutionSample(x=array([1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1.]), fval=-679.0, probability=0.0019531250000000004, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1.]), fval=-679.0, probability=0.0009765625, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1.]), fval=-653.0, probability=0.0009765625, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 1.]), fval=-652.0, probability=0.0019531250000000004, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0.]), fval=-637.0, probability=0.0009765625, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0.]), fval=-636.0, probability=0.0019531250000000004,